<p style="font-family: Arial; font-size:3.0em; color:purple; font-style:bold">
<br>SHOWING FANAL Kr-ANALYSIS RESULTS<br></p>

This notebook illustrates the results got from FANAL 'kr' analysis.

It provides contents of krypton DataFrames with some associated plots.

#### Setting general stuff

In [ ]:
from IPython.core.display import HTML
css = open('css/style-table.css').read() + open('css/style-notebook.css').read()
HTML('<style>{}</style>'.format(css))

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%matplotlib inline

#### Importings

In [ ]:
import os
import sys
import numpy  as np
import tables as tb
import pandas as pd
import matplotlib.pyplot as plt
from   matplotlib.colors import LogNorm

#%matplotlib widget

In [ ]:
# Specific IC stuff

import invisible_cities.core.system_of_units  as units

In [ ]:
# Specific FANAL stuff
from fanal.core.fanal_units import Qbb
from fanal.core.fanal_units import kr_energy
from fanal.core.fanal_units import xe_Wsci

from fanal.core.detectors   import get_detector

from fanal.utils.types      import XYZ

from fanal.utils.plot_utils import plot_vertices
from fanal.utils.plot_utils import plot_and_fit

In [ ]:
plt.rcParams["figure.figsize"] = 8, 6
plt.rcParams["font.size"     ] = 14

# LOADING DATA

In [ ]:
iPATH         = "../data/flex100f"
results_fname = iPATH + "/kr/fanal/fanal.flex100f.kr.h5"

In [ ]:
results_fname = "/Users/Javi/Development/FANAL/fanal/tmp/fanal.FLEX100_M2_O6.kr.h5"
#results_fname = "/Users/Javi/Development/FANAL/fanal/tmp/tst.kr.h5"

In [ ]:
config_df  = pd.read_hdf(results_fname, "FANAL" + '/config')
kryton_df  = pd.read_hdf(results_fname, "FANAL" + '/kryptons')
#results_df = pd.read_hdf(results_fname, "FANAL" + '/results')

In [ ]:
print("Configuration run ...")
config_df

In [ ]:
detector = get_detector(config_df.loc['det_name'].value)

In [ ]:
detector

In [ ]:
kryton_df.head()

In [ ]:
#results_df

# VERTICES

In [ ]:
plot_vertices(kryton_df.rename(columns = {'x_true': 'x', 'y_true': 'y', 'z_true': 'z'}),
              num_bins=40, extra_size = 50)

# POSITIONS

In [ ]:
dist_x  = kryton_df.x_true - kryton_df.x_rec
dist_y  = kryton_df.y_true - kryton_df.y_rec

In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2, figsize=(16,7))
ax1.hist(dist_x, 50);
ax1.set_title ("Distance X", size=16);
ax1.set_xlabel("mm", size=16);
ax1.set_ylabel("entries", size=16);

energy_sns_pde = 0.4
ax2.hist(dist_y, 50);
ax2.set_title (f"Distance Y", size=16);
ax2.set_xlabel("mm", size=16);
ax2.set_ylabel("entries", size=16);

print("Distance X description:")
print(dist_x.describe())
print("Distance Y description:")
print(dist_y.describe())

In [ ]:
#dist_xy = XYZ(kryton_df.x_true, kryton_df.y_true, 0.).distance( \
#          XYZ(kryton_df.x_rec , kryton_df.y_rec , 0.))
#
#plt.hist(dist_xy, bins = 100);
#print("Distance XY description:")
#dist_xy.describe()

# ENERGY - S1

In [ ]:
kryton_df.s1_pes.describe()

In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2, figsize=(16,7))
ax1.hist(kryton_df.s1_pes, 50, range=[0, 100]);
ax1.set_title ("S1 (raw)", size=16);
ax1.set_xlabel("pes"     , size=16);
ax1.set_ylabel("entries" , size=16);

ax2.hist2d(kryton_df.rad_true, kryton_df.s1_pes, (50, 50), range=[[20, 470], [0, 100]]);
ax2.set_title ("S1 (raw)"   , size=16);
ax2.set_xlabel("Radius (mm)", size=16);
ax2.set_ylabel("pes"        , size=16);

# Light Collection Efficiency

In [ ]:
# The number of initial scintillation photons for kr is 
# the kr energy / Wsci del xenon
num_kr_s1_photons = kr_energy / xe_Wsci

# This collection efficiency has been computed asuming that the sesnors used to read it:
# Have a pde = 100 %
# Don't have background
kryton_df['col_eff'] = kryton_df.s1_pes / num_kr_s1_photons / units.perCent

In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2, figsize=(16,7))
ax1.hist(kryton_df.col_eff, 50, range=(0, 15));
ax1.set_title ("Light Collection Efficiency", size=16);
ax1.set_xlabel("Eff (%)", size=16);
ax1.set_ylabel("entries", size=16);

energy_sns_pde = 0.4
ax2.hist(kryton_df.col_eff * energy_sns_pde, 50, range=(0, 6));
ax2.set_title (f"Light Collection Efficiency (sns pde = {energy_sns_pde})", size=16);
ax2.set_xlabel("Eff (%)", size=16);
ax2.set_ylabel("entries", size=16);

(kryton_df.col_eff * energy_sns_pde).describe()

# ENERGY - S2

In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2, figsize=(16,7))
#ax1.hist(kryton_df.s2_pes, 50)
ax1.hist(kryton_df.s2_pes, 50, (10000, 20000))
ax1.set_title ("S2 (raw)"   , size=16);
ax1.set_xlabel("pes"     , size=16);
ax1.set_ylabel("entries" , size=16);

ax2.hist2d(kryton_df.rad_true, kryton_df.s2_pes, (50, 50), range=[[20, 470], [10000, 20000]])
ax2.set_title ("S2 (raw)"   , size=16);
ax2.set_xlabel("Radius (mm)", size=16);
ax2.set_ylabel("pes"        , size=16);

In [ ]:
mu, sigma, fwhm = plot_and_fit(kryton_df.s2_pes_corr, title="corrected S2", num_bins=100);

print(f"fwhm at Qbb: {fwhm * (kr_energy / Qbb)**.5 / units.perCent:.3} %")

In [ ]:
mu, sigma, fwhm = plot_and_fit(kryton_df.energy_rec, title="Energy reconstructed", num_bins=100);

print(f"fwhm at Qbb: {fwhm * (kr_energy / Qbb)**.5 / units.perCent:.3} %")

# Charge

In [ ]:
print(f"Current detector:\n {detector}\n")

In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2, figsize=(16,7))
ax1.hist(kryton_df.q_tot, 50)
#ax1.hist(kryton_df.q_tot, 50, range=[900, 1300])
ax1.set_title ("Q tot"   , size=16);
ax1.set_xlabel("pes"     , size=16);
ax1.set_ylabel("entries" , size=16);

ax2.hist(kryton_df.q_max, 50)
ax2.set_title ("Q max"   , size=16);
ax2.set_xlabel("pes"     , size=16);
ax2.set_ylabel("entries" , size=16);

In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2, figsize=(16,7))
ax1.hist2d(kryton_df.rad_true, kryton_df.q_tot, (50, 50))
#ax1.hist2d(kryton_df.rad_true, kryton_df.q_tot, (50, 50), range=[[20, 470], [900, 1300]])
ax1.set_title ("Q tot"      , size=16);
ax1.set_xlabel("Radius (mm)", size=16);
ax1.set_ylabel("pes"        , size=16);

#ax2.hist2d(kryton_df.rad_true, kryton_df.q_max, (50, 50))
ax2.hist2d(kryton_df.rad_true, kryton_df.q_max, (50, 50), range=[[20, 470], [0, 100]])
ax2.set_title ("Q max"      , size=16);
ax2.set_xlabel("Radius (mm)", size=16);
ax2.set_ylabel("pes"        , size=16);